In [ ]:
# Install Kaggle package
!pip install kaggle

In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a directory named 'kaggle' and copy kaggle.json file for authentication
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json  # Set file permissions

In [ ]:

# Download the dataset
!kaggle datasets download -d arashnic/book-recommendation-dataset

Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
License(s): CC0-1.0
 37% 9.00M/24.3M [00:00<00:00, 57.5MB/s]
100% 24.3M/24.3M [00:00<00:00, 116MB/s] 


In [ ]:

# Unzip the dataset
!unzip book-recommendation-dataset

Archive:  book-recommendation-dataset.zip
  inflating: Books.csv               
  inflating: DeepRec.png             
  inflating: Ratings.csv             
  inflating: Users.csv               
  inflating: classicRec.png          
  inflating: recsys_taxonomy2.png    


In [ ]:
import pandas as pd


In [ ]:
# Read dataset files
books = pd.read_csv('Books.csv')

<ipython-input-5-4806dc8eda5b>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [ ]:
print(books.head())

         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/

In [ ]:
users = pd.read_csv('Users.csv')

In [ ]:
print(users.head())

   User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN


In [ ]:
ratings = pd.read_csv('Ratings.csv')

In [ ]:
print(ratings.head())

   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6


In [ ]:
# Merge datasets based on common column (ISBN)
merged_data = pd.merge(ratings, books, on='ISBN')
merged_data = pd.merge(merged_data, users, on='User-ID')
print(merged_data.head())

   User-ID        ISBN  Book-Rating  \
0   276725  034545104X            0   
1     2313  034545104X            5   
2     2313  0812533550            9   
3     2313  0679745580            8   
4     2313  0060173289            9   

                                         Book-Title       Book-Author  \
0                              Flesh Tones: A Novel        M. J. Rose   
1                              Flesh Tones: A Novel        M. J. Rose   
2     Ender's Game (Ender Wiggins Saga (Paperback))  Orson Scott Card   
3             In Cold Blood (Vintage International)     TRUMAN CAPOTE   
4  Divine Secrets of the Ya-Ya Sisterhood : A Novel     Rebecca Wells   

  Year-Of-Publication         Publisher  \
0                2002  Ballantine Books   
1                2002  Ballantine Books   
2                1986         Tor Books   
3                1994           Vintage   
4                1996     HarperCollins   

                                         Image-URL-S  \
0  http://i

In [ ]:
# Handle missing values
merged_data['Age'].fillna(merged_data['Age'].mean(), inplace=True)
merged_data['Location'].fillna('Unknown', inplace=True)  # Fill missing Location with 'Unknown'
merged_data['Publisher'].fillna('Unknown', inplace=True) # Fill missing Publisher with 'Unknown'


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()


In [ ]:

print(merged_data.head())

   User-ID        ISBN  Book-Rating  \
0   276725  034545104X            0   
1     2313  034545104X            5   
2     2313  0812533550            9   
3     2313  0679745580            8   
4     2313  0060173289            9   

                                         Book-Title       Book-Author  \
0                              Flesh Tones: A Novel        M. J. Rose   
1                              Flesh Tones: A Novel        M. J. Rose   
2     Ender's Game (Ender Wiggins Saga (Paperback))  Orson Scott Card   
3             In Cold Blood (Vintage International)     TRUMAN CAPOTE   
4  Divine Secrets of the Ya-Ya Sisterhood : A Novel     Rebecca Wells   

  Year-Of-Publication         Publisher  \
0                2002  Ballantine Books   
1                2002  Ballantine Books   
2                1986         Tor Books   
3                1994           Vintage   
4                1996     HarperCollins   

                                         Image-URL-S  \
0  http://i

In [ ]:
# Encode 'ISBN' and 'User-ID' using Label Encoding
merged_data['ISBN'] = label_encoder.fit_transform(merged_data['ISBN'])
merged_data['User-ID'] = label_encoder.fit_transform(merged_data['User-ID'])
# Perform One-Hot Encoding for 'Location' and 'Publisher'
merged_data = pd.get_dummies(merged_data, columns=['Location', 'Publisher'])


NameError: name 'label_encoder' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)
print("Number of training samples:", len(train_data))
print("Number of testing samples:", len(test_data))

In [ ]:
def hide_ratings(data):
    # Filter users with fewer than 3 ratings
    users_with_enough_ratings = data['User-ID'].value_counts()[data['User-ID'].value_counts() >= 3].index
    data_filtered = data[data['User-ID'].isin(users_with_enough_ratings)]
    # Sample 3 items for each user
    hidden_ratings = data_filtered.groupby('User-ID').apply(lambda x: x.sample(n=3, random_state=42))
    visible_ratings = data_filtered[~data_filtered.index.isin(hidden_ratings.index)]
    return visible_ratings, hidden_ratings

train_visible, train_hidden = hide_ratings(train_data)
test_visible, test_hidden = hide_ratings(test_data)

print("Number of visible training samples:", len(train_visible))
print("Number of hidden training samples:", len(train_hidden))
print("Number of visible testing samples:", len(test_visible))
print("Number of hidden testing samples:", len(test_hidden))


In [ ]:
import numpy as np

# Filter users who have rated 7 or more books
user_ratings_count = merged_data['User-ID'].value_counts()
users_with_enough_ratings = user_ratings_count[user_ratings_count >= 7].index
filtered_data = merged_data[merged_data['User-ID'].isin(users_with_enough_ratings)]

# Randomly select 100 users
selected_users = np.random.choice(users_with_enough_ratings, size=100, replace=False)

# Split data for selected users into visible and hidden samples
visible_data = filtered_data[filtered_data['User-ID'].isin(selected_users)].copy()
hidden_data = visible_data.groupby('User-ID').apply(lambda x: x.sample(n=3, random_state=42)).reset_index(drop=True)
visible_data = visible_data[~visible_data.index.isin(hidden_data.index)]

print("Number of visible samples:", len(visible_data))
print("Number of hidden samples:", len(hidden_data))

In [ ]:
from sklearn.preprocessing import OneHotEncoder

isbn_encoder = OneHotEncoder(sparse=False)
user_encoder = OneHotEncoder(sparse=False)

X_books = isbn_encoder.fit_transform(visible_data[['ISBN']])
X_users = user_encoder.fit_transform(visible_data[['User-ID']])

X = np.concatenate([X_books, X_users], axis=1)

In [ ]:
# Number of features in one-hot encoded data
num_features_books = X_books.shape[1]
num_features_users = X_users.shape[1]
total_features = num_features_books + num_features_users
print("Total number of features:", total_features)


In [ ]:
# 1. Verify Input Data Dimensions
print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)

# 2. Check Model Input Shape
print("Expected input shape for the model:", model.input_shape)

# 3. Inspect Data Preprocessing
# Ensure that your data preprocessing steps do not alter the input dimensions unexpectedly.
# If you're using any transformations or scaling, verify that they preserve the shape of the input data.

# 4. Check for Data Integrity
# Check for missing values or anomalies in your data that could affect input dimensions.
print("Number of missing values in training data:", np.isnan(train_data).sum())
print("Number of missing values in testing data:", np.isnan(test_data).sum())


In [ ]:
# Concatenate visible and hidden data for fitting the encoder
all_data = pd.concat([visible_data, hidden_data_grouped])

# One-hot encode categorical features
X_books_all = isbn_encoder.fit_transform(all_data[['ISBN']])
X_users_all = user_encoder.fit_transform(all_data[['User-ID']])

# Split the one-hot encoded data back into visible and hidden parts
X_books_visible = X_books_all[:len(visible_data)]
X_books_hidden = X_books_all[len(visible_data):]
X_users_visible = X_users_all[:len(visible_data)]
X_users_hidden = X_users_all[len(visible_data):]

# Concatenate the encoded features
X_visible = np.concatenate([X_books_visible, X_users_visible], axis=1)
X_hidden = np.concatenate([X_books_hidden, X_users_hidden], axis=1)

# Predict ratings for the hidden data
hidden_y_pred = model.predict(X_hidden)

# Compute RMSE and MSE
rmse = np.sqrt(mean_squared_error(hidden_ratings, hidden_y_pred))
mse = mean_squared_error(hidden_ratings, hidden_y_pred)

print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Squared Error (MSE):", mse)


In [ ]:

from keras.models import Sequential
from keras.layers import Dense

input_shape = X.shape[1]

model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mean_squared_error')

y = visible_data['Book-Rating']

model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

hidden_data_grouped = hidden_data.groupby('User-ID').head(3)

hidden_X_books = isbn_encoder.transform(hidden_data_grouped[['ISBN']])
hidden_X_users = user_encoder.transform(hidden_data_grouped[['User-ID']])
hidden_X = np.concatenate([hidden_X_books, hidden_X_users], axis=1)
hidden_y_pred = model.predict(hidden_X)

hidden_ratings = hidden_data_grouped['Book-Rating'].values

comparison_df = pd.DataFrame({'User-ID': hidden_data_grouped['User-ID'], 'Hidden Rating': hidden_ratings, 'Predicted Rating': hidden_y_pred.flatten()})
print(comparison_df)